In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:2]:
        print(os.path.join(dirname, filename))

In [2]:
import os
os.makedirs('/kaggle/working/data')

os.makedirs('/kaggle/working/data/train')
os.makedirs('/kaggle/working/data/val')
os.makedirs('/kaggle/working/data/test')

os.makedirs('/kaggle/working/data/train/dogs')
os.makedirs('/kaggle/working/data/train/cats')

os.makedirs('/kaggle/working/data/test/dogs')
os.makedirs('/kaggle/working/data/test/cats')

os.makedirs('/kaggle/working/data/val/dogs')
os.makedirs('/kaggle/working/data/val/cats')

os.makedirs('/kaggle/working/model')

In [3]:
import shutil

def move(source, dest, half=""):
    source_file_lst = os.listdir(source)
    
    if half == "left":
        source_file_lst = source_file_lst[:len(source_file_lst)//2]
    elif half == "right":
        source_file_lst = source_file_lst[len(source_file_lst)//2:]
        
    for f in source_file_lst:
        shutil.copy(source + f, dest)
    
    print(str(len(source_file_lst)) + " files copied from " + source + " to " + dest)

In [4]:
move('/kaggle/input/cat-and-dog/training_set/training_set/dogs/', '/kaggle/working/data/train/dogs')
move('/kaggle/input/cat-and-dog/training_set/training_set/cats/', '/kaggle/working/data/train/cats')

move('/kaggle/input/cat-and-dog/test_set/test_set/dogs/', '/kaggle/working/data/test/dogs', half="left")
move('/kaggle/input/cat-and-dog/test_set/test_set/cats/', '/kaggle/working/data/test/cats', half="left")

move('/kaggle/input/cat-and-dog/test_set/test_set/dogs/', '/kaggle/working/data/val/dogs', half="right")
move('/kaggle/input/cat-and-dog/test_set/test_set/cats/', '/kaggle/working/data/val/cats', half="right")

In [7]:
from os import path

#path
DATA_PATH = "/kaggle/working/data/"

TRAIN_DIR = path.join(DATA_PATH, 'train')
VAL_DIR = path.join(DATA_PATH, 'val')
TEST_DIR = path.join(DATA_PATH, 'test')

MODEL_DIR = "/kaggle/working/model/"

In [5]:
from tensorflow.keras.applications import Xception 

base_model = Xception(input_shape=(299, 299, 3), include_top=False)
base_model.trainable = False 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([base_model, Flatten(), Dense(2, activation='softmax', name="out")]) 

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20, 
                                    zoom_range=0.1,
                                   horizontal_flip=True)

train_generator = train_data_gen.flow_from_directory(TRAIN_DIR, 
                                                     target_size=(299, 299),
                                                     color_mode='rgb',
                                                     batch_size =128,
                                                     class_mode='categorical',
                                                     shuffle=True)         

val_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data_gen.flow_from_directory(VAL_DIR, 
                                                target_size=(299, 299),
                                                color_mode='rgb',
                                                batch_size =64,
                                                class_mode='categorical',
                                                shuffle=True)         


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint(MODEL_DIR, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [10]:
model.fit(train_generator, epochs=2, validation_data=val_generator, callbacks=[checkpoint], verbose=1)

In [17]:
from tensorflow import uint8, float32, cast
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

i = Input([299, 299, 3], dtype=uint8)
x = cast(i, float32)
x = preprocess_input(x)
x = model(x)
runtime_model = Model(inputs=[i], outputs=[x])

In [19]:
RUNTIME_MODEL_DIR = path.join(MODEL_DIR, 'runtime')
runtime_model.save(RUNTIME_MODEL_DIR)

In [20]:
from tensorflow.keras.models import load_model 

loaded_model = load_model(RUNTIME_MODEL_DIR)


In [24]:
test_data_gen = ImageDataGenerator()

test_generator = test_data_gen.flow_from_directory(TEST_DIR, 
                                                   target_size=(299, 299),
                                                   color_mode='rgb',
                                                   batch_size =128,
                                                   class_mode='categorical',
                                                   shuffle=False)  


In [26]:
raw_preds = loaded_model.predict(test_generator)
raw_preds

In [32]:
import numpy as np
np.argmax(raw_preds, axis=1)

def predict(model = loaded_model, generator=test_generator):
    max_indices = np.argmax(model.predict(generator), axis=1).tolist()
    return ["Cat" if i == 0 else "Dog" for i in max_indices]

In [33]:
predict()